In [ ]:
import pandas as pd
df = pd.read_csv("chicagocrimes.csv")
df.head()


/tmp/ipython-input-3747521720.py:2: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("chicagocrimes.csv")


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09-05-2015 13:30,043XX S WOOD ST,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015.0,02-10-2018 15:50,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09-04-2015 11:30,008XX N CENTRAL AVE,870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,6,1138875.0,1904869.0,2015.0,02-10-2018 15:50,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09-01-2018 00:01,082XX S INGLESIDE AVE,810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,6,NaN,NaN,2018.0,04-06-2019 16:04,NaN,NaN,NaN
3,10224740,HY411595,09-05-2015 12:45,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015.0,02-10-2018 15:50,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09-05-2015 13:00,0000X N LARAMIE AVE,560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015.0,02-10-2018 15:50,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [ ]:
df.shape



(51210, 22)

In [ ]:
df = df[[
    "Date",
    "Block",
    "Primary Type",
    "District",
    "Community Area",
    "Year",
    "Latitude",
    "Longitude"
]]


In [ ]:
df.columns = [
    "date",
    "block",
    "primary_type",
    "district",
    "community_area",
    "year",
    "latitude",
    "longitude"
]


In [ ]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51210 entries, 0 to 51209
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            22649 non-null  datetime64[ns]
 1   block           51210 non-null  object        
 2   primary_type    51209 non-null  object        
 3   district        51209 non-null  float64       
 4   community_area  51205 non-null  float64       
 5   year            51209 non-null  float64       
 6   latitude        50164 non-null  float64       
 7   longitude       50164 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 3.1+ MB


In [ ]:
df = df.dropna(subset=["latitude", "longitude"])


In [ ]:
df["year"] = df["year"].astype(int)


In [ ]:
df = df.dropna(subset=["date"])


In [ ]:
df.info()
df.shape


<class 'pandas.core.frame.DataFrame'>
Index: 22086 entries, 0 to 51208
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            22086 non-null  datetime64[ns]
 1   block           22086 non-null  object        
 2   primary_type    22086 non-null  object        
 3   district        22086 non-null  float64       
 4   community_area  22086 non-null  float64       
 5   year            22086 non-null  int64         
 6   latitude        22086 non-null  float64       
 7   longitude       22086 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 1.5+ MB


(22086, 8)

In [ ]:
df.to_csv("chicago_crime_2020_present_clean.csv", index=False)


In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN


In [ ]:
coords = df[["latitude", "longitude"]].to_numpy()
coords.shape



(22086, 2)

In [ ]:
coords_rad = np.radians(coords)


In [ ]:
dbscan = DBSCAN(
    eps=0.01,        # distance threshold (~1 km)
    min_samples=20,  # minimum crimes to form a hotspot
    metric="haversine"
)

df["hotspot"] = dbscan.fit_predict(coords_rad)


In [ ]:
df["hotspot"].value_counts()


,count
hotspot,
0,22086


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

coords = df[["latitude", "longitude"]].to_numpy()
coords_rad = np.radians(coords)

dbscan = DBSCAN(
    eps=0.00015,     # ~1 km
    min_samples=20,
    metric="haversine"
)

df["hotspot"] = dbscan.fit_predict(coords_rad)


In [ ]:
df["hotspot"].value_counts().head(10)


,count
hotspot,
0,21923
1,87
-1,76


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

coords = df[["latitude", "longitude"]].to_numpy()
coords_rad = np.radians(coords)

dbscan = DBSCAN(
    eps=0.00008,     # ~500 meters
    min_samples=25,  # slightly stricter
    metric="haversine"
)

df["hotspot"] = dbscan.fit_predict(coords_rad)


In [ ]:
df["hotspot"].value_counts().head(10)


,count
hotspot,
0,19870
-1,1435
4,99
1,85
6,84
5,79
2,65
8,59
12,53


In [ ]:
hotspot_df = df[df["hotspot"] > 0].copy()


In [ ]:
hotspot_df.shape


(781, 9)

In [ ]:
zone_summary = hotspot_df.groupby("hotspot").agg(
    crime_count=("hotspot", "count"),
    avg_latitude=("latitude", "mean"),
    avg_longitude=("longitude", "mean"),
    main_crime_type=("primary_type", lambda x: x.value_counts().idxmax()),
    main_district=("district", lambda x: x.value_counts().idxmax()),
    main_community=("community_area", lambda x: x.value_counts().idxmax())
).reset_index()


,hotspot,crime_count,avg_latitude,avg_longitude,main_crime_type,main_district,main_community
0,1,85,41.713030,-87.535533,BATTERY,4.0,52.0
1,2,65,41.944889,-87.686786,THEFT,19.0,5.0
2,3,27,41.754641,-87.741530,THEFT,8.0,65.0
3,4,99,41.656120,-87.605996,BATTERY,5.0,54.0
4,5,79,41.967499,-87.764097,BATTERY,16.0,15.0
